<a href="https://colab.research.google.com/github/PatilAmartyaJ/NewsMiner/blob/main/Newspaper_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def getUrl(location,date_obj):
   url='https://epaper.lokmat.com/main-editions/'+location
   url+="%20Main/"
   url+= str(date_obj)
   url+="/"
   url+="1"
   print(url)
   return url




In [ ]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

def getArticles(url):
    # Setup Chrome in headless mode
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)


        # Wait until the element with debug-id="slide-0" is present
        wait = WebDriverWait(driver, 1000)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[debug-id="slide-0"]')))

        # Locate the element and print its text
        container = driver.find_element(By.CSS_SELECTOR, '[debug-id="slide-0"]')
        links = container.find_elements(By.TAG_NAME, 'a')
        return len(links)

    except (ConnectionRefusedError, WebDriverException) as e:

            if 'driver' in locals():
                driver.quit()
            # time.sleep(2 ** attempt)  # Exponential backoff

    finally:
        driver.quit()



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
# Set up Chrome options without user data dir
options = Options()
options.add_argument("--headless")  # Run in headless mode (no GUI)
options.add_argument("--disable-gpu")  # Recommended for headless
options.add_argument("--no-sandbox")  # Good for Linux
options.add_argument("--disable-dev-shm-usage")  # Prevent resource issues

# Make absolutely sure we're NOT reusing a user profile
# (NO --user-data-dir is added here)

# Create driver
driver = webdriver.Chrome(options=options)

def getPageCount(url):
    try:
    # Target URL
      driver.get(url)

    # Wait for the <ul> element to have content
      wait = WebDriverWait(driver, 1000)
      wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul#mTS_1_container li")))

    # Extract inner HTML
      container = driver.find_element(By.ID, "mTS_1_container")
      html_content = container.get_attribute("innerHTML")

      soup = BeautifulSoup(html_content, 'html.parser')

# Find all <li> elements
      li_elements = soup.find_all('li')
      print(len(li_elements))
      return len(li_elements)

    finally:
      driver.quit()




In [ ]:
import os
import signal

def kill_chrome_processes():
    try:
        # Kill any existing Chrome processes
        os.system("pkill -f chrome")
        os.system("pkill -f chromedriver")
    except:
        pass

# Call this before starting your scraping
kill_chrome_processes()

In [ ]:
import requests
import os
from datetime import datetime



# Image URL (replace with your target image)
location_dict={
    "Kolhapur":"KOLK",
    "Mumbai": "MULK",
    "Pune":"PUNK",
    "Nagpur":"NPLK",
    "Nashik":"NSLK",
    "Chhatrapati Sambhajinagar":"AULK",
    "Jalgoan":"JLLK",
    "Akola":"AKLK",
    "Solapur":"SOLK",
    "Ahilyanagar":"ANLK",
    "Goa":"GALK",
    "New Delhi":"DLLK"
}
print("Valid locations are: ")
for key in location_dict.keys():
    print(key)
location=input("Enter the Location: ")


date_str = input("Enter a date (YYYY-MM-DD): ")
date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

print("You entered:", date_obj)

invalid_dates=["2025-01-27","2025-03-14","2025-05-02","2025-08-16"]
url_prefix = 'https://epaper.lokmat.com/'
if date_obj.strftime("%Y-%m-%d") not in invalid_dates and date_obj<=datetime.now().date():
   url= getUrl(location,date_obj)
   number_of_pages=getPageCount(url)
   clean_url = url.rsplit('/', 1)[0]
   print(clean_url)
   for i in range(1,number_of_pages+1):
       url_page=clean_url+"/"+str(i)
       print(url_page)
       num_articles=getArticles(url_page)
       print("number of articles for page number ",i," are ",num_articles)
       for j in range(1,num_articles+1):
          article_url=url_prefix+"articlepage.php?articleid=LOK_"
          article_url+=location_dict[location]+"_"
          article_url+=  date_obj.strftime("%Y%m%d")
          article_url+="_"+str(i)+"_"+str(j)
          print(article_url)



# https://epaper.lokmat.com/articlepage.php?articleid=LOK_KOLK_20250828
# 'KOLK/2025/08/24/ArticleImages/68aa3de90ef42.jpg'

# # Extract the filename from the URL
# filename = os.path.basename(image_url)

# # Make the request to get the image content
# response = requests.get(image_url)

# if response.status_code == 200:
#     with open(filename, 'wb') as f:
#         f.write(response.content)
#     print(f"Image saved as: {filename}")
# else:
#     print(f"Failed to download image. Status code: {response.status_code}")


Valid locations are: 
Kolhapur
Mumbai
Pune
Nagpur
Nashik
Chhatrapati Sambhajinagar
Jalgoan
Akola
Solapur
Ahilyanagar
Goa
New Delhi
Enter the Location: Kolhapur
Enter a date (YYYY-MM-DD): 2025-08-07
You entered: 2025-08-07
https://epaper.lokmat.com/main-editions/Kolhapur%20Main/2025-08-07/1
12
https://epaper.lokmat.com/main-editions/Kolhapur%20Main/2025-08-07
https://epaper.lokmat.com/main-editions/Kolhapur%20Main/2025-08-07/1
number of articles for page number  1  are  2
https://epaper.lokmat.com/articlepage.php?articleid=LOK_KOLK_20250807_1_1
https://epaper.lokmat.com/articlepage.php?articleid=LOK_KOLK_20250807_1_2
https://epaper.lokmat.com/main-editions/Kolhapur%20Main/2025-08-07/2
number of articles for page number  2  are  2
https://epaper.lokmat.com/articlepage.php?articleid=LOK_KOLK_20250807_2_1
https://epaper.lokmat.com/articlepage.php?articleid=LOK_KOLK_20250807_2_2
https://epaper.lokmat.com/main-editions/Kolhapur%20Main/2025-08-07/3
number of articles for page number  3  are  

KeyboardInterrupt: 